In [1]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
from steam import WebAPI
from datetime import datetime
import os
import glob
import json
import tensorflow as tf
import sys
from IPython import display
import requests
import tensorflow.keras.backend as K
from sklearn import decomposition

#import logging
#logging.getLogger("tensorflow").setLevel(logging.ERROR)

#from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()

print(tf.__version__)

2.0.0


In [2]:
sns.set_style('ticks')

In [3]:
champfile=os.path.join('lol_data','version9.10.1_champion.json')
with open(champfile,'r') as fin:
    champdata=json.load(fin)
champ_vocab=[]
for name,data in champdata['data'].items():
    champ_vocab.append(data['key'])
champ_vocab = [int(i) for i in champ_vocab]
print(len(champ_vocab))

144


# Modeling using a TFRecord dataformat

## Build TFRecord input pipeline

In [16]:
def _parse_function_extra_info(example_proto):
    """Extracts features and labels.
  
    Args:
        example_proto: tf.Example protocol (unsure what this is)    
      Returns:
    A `tuple` `(labels, features)`:
      features: A dict of tensors representing the features
      labels: A tensor with the corresponding labels.
    """
    features = {}
    
    #loop through five times to capture each hero on each team
    for i in range(5):
        #64 bit int for hero
        features["radiant_hero"+str(i)] =tf.io.FixedLenFeature(1, dtype=tf.int64)
        #there are 4 pieces of info for a hero in LOL
        features["radiant_hero_info"+str(i)] =tf.io.FixedLenFeature(4, dtype=tf.int64) 
        #there are 6 possible tags for a hero in LOL
        features["radiant_hero_tags"+str(i)] =tf.io.FixedLenFeature(6, dtype=tf.int64) 
        features["dire_hero"+str(i)] =tf.io.FixedLenFeature(1, dtype=tf.int64) 
        features["dire_hero_info"+str(i)] =tf.io.FixedLenFeature(4, dtype=tf.int64) 
        features["dire_hero_tags"+str(i)] =tf.io.FixedLenFeature(6, dtype=tf.int64) 
 
    features["skill_level"] = tf.io.FixedLenFeature(2, dtype=tf.int64)
    features["targets"] = tf.io.FixedLenFeature(1, dtype=tf.int64)
                
    parsed_features = tf.io.parse_single_example(example_proto, features)
    
    hero_dict={}
    for i in range(5):
        hero_dict['radiant_hero'+str(i)]=parsed_features['radiant_hero'+str(i)]
        hero_dict['radiant_hero_info'+str(i)]=parsed_features['radiant_hero_info'+str(i)]
        hero_dict['radiant_hero_tags'+str(i)]=parsed_features['radiant_hero_tags'+str(i)]
        hero_dict['dire_hero'+str(i)]=parsed_features['dire_hero'+str(i)]
        hero_dict['dire_hero_info'+str(i)]=parsed_features['dire_hero_info'+str(i)]
        hero_dict['dire_hero_tags'+str(i)]=parsed_features['dire_hero_tags'+str(i)]

    hero_dict['skill_level']= parsed_features['skill_level']
    target = parsed_features['targets']
    
    return hero_dict, target

#### Check the parse function worked

In [17]:
# Create the Dataset object.
ds = tf.data.TFRecordDataset(os.path.join('LOL_data',
                                          'mix_tier',
                                          'lol_training_data',
                                          'blue_win_extra_info_dota2_compat_lol_training_data.tfrecords'),
                                          compression_type="GZIP")
# Map features and labels with the parse function.
ds = ds.map(_parse_function_extra_info)

In [18]:
ds = ds.shuffle(10000)
print(next(iter(ds)))

({'radiant_hero0': <tf.Tensor: id=1085, shape=(1,), dtype=int64, numpy=array([45])>, 'radiant_hero_info0': <tf.Tensor: id=1090, shape=(4,), dtype=int64, numpy=array([7, 6, 2, 5])>, 'radiant_hero_tags0': <tf.Tensor: id=1095, shape=(6,), dtype=int64, numpy=array([0, 1, 0, 0, 0, 1])>, 'dire_hero0': <tf.Tensor: id=1070, shape=(1,), dtype=int64, numpy=array([38])>, 'dire_hero_info0': <tf.Tensor: id=1075, shape=(4,), dtype=int64, numpy=array([7, 4, 5, 5])>, 'dire_hero_tags0': <tf.Tensor: id=1080, shape=(6,), dtype=int64, numpy=array([1, 1, 0, 0, 0, 0])>, 'radiant_hero1': <tf.Tensor: id=1086, shape=(1,), dtype=int64, numpy=array([86])>, 'radiant_hero_info1': <tf.Tensor: id=1091, shape=(4,), dtype=int64, numpy=array([3, 4, 8, 5])>, 'radiant_hero_tags1': <tf.Tensor: id=1096, shape=(6,), dtype=int64, numpy=array([1, 0, 1, 0, 0, 0])>, 'dire_hero1': <tf.Tensor: id=1071, shape=(1,), dtype=int64, numpy=array([9])>, 'dire_hero_info1': <tf.Tensor: id=1076, shape=(4,), dtype=int64, numpy=array([6, 6, 7

## Model Functions

In [15]:
# Create an input_fn that parses the tf.Examples from the given files,
# and split them into features and targets.
def _input_fn_extra_info(input_filenames, num_epochs=None, 
              shuffle=True, batch_size=50,compression_type=""):
   
   # Same code as above; create a dataset and map features and labels.
    ds = tf.data.TFRecordDataset(input_filenames,compression_type=compression_type)
    ds = ds.map(_parse_function_extra_info)

    if shuffle:
        ds = ds.shuffle(10000)
    ds = ds.batch(batch_size)
    ds = ds.repeat()
    
    # Return the dataset.
    return ds

## Define and train the linear model

### various iterations of linear models

In [24]:
#define the linear model with no feature columns
def define_linear_model_extra_info(vocab_len, 
                                   embedding_dims):
    '''Function to define simple keras linear model
    
    Args: 
        vocab_len: An `int` specifying the number of category of the categorical input
        embedding_dims: An `int` specifying the number of dimensions in the embedding layer
    Returns:
        model: A uncompiled `tf.keras.model`
    '''
    
    ##define hero processing layers
    #embed hero
    embed_hero = tf.keras.layers.Embedding(input_dim=vocab_len,
                                           output_dim=embedding_dims)
    #process tags into num of values specified by dense outputs value
    dense_hero_tags = tf.keras.layers.Dense(1,activation='linear')
    #process info into num of values specified by dense outputs value
    dense_hero_info = tf.keras.layers.Dense(1,activation='linear')
    ##process hero embed, dense_tags, and dense_info output into some num of outputs
    #dense_hero_overall = tf.keras.Dense(embedding_dims+2,activation='linear')
    
    hero_input=[]
    rad_hero_output=[]
    dire_hero_output=[]
    #process all heroes in same way
    for i in range(5):
        #add radiant hero to hero_input list and process with
        #    embed_hero layer
        hero_input.append(tf.keras.Input(shape=(1,),
                                         name='radiant_hero'+str(i)))
        x_rh = embed_hero(hero_input[-1])
        x_rh = tf.keras.layers.Flatten()(x_rh)
        #add radiant hero info to hero_input list and process with
        #    dense_tags layer
        hero_input.append(tf.keras.Input(shape=(4,), #4 info types
                                         name='radiant_hero_info'+str(i)))
        x_rhi = dense_hero_info(hero_input[-1])
        #add radiant hero tags to hero_input list and process with
        #    dense_info layer
        hero_input.append(tf.keras.Input(shape=(6,), #6 tag types
                                         name='radiant_hero_tags'+str(i)))
        x_rht = dense_hero_tags(hero_input[-1])
        #append output of hero layers to rad_out_layer for summing later
        rad_hero_output.append(tf.keras.layers.concatenate([x_rh,x_rhi,x_rht]))
        

        #add dire hero to hero_input list and process with
        #    embed_hero layer
        hero_input.append(tf.keras.Input(shape=(1,),
                                         name='dire_hero'+str(i)))
        x_dh = embed_hero(hero_input[-1])
        x_dh = tf.keras.layers.Flatten()(x_dh)
        #add dire hero info to hero_input list and process with
        #    dense_info layer       
        hero_input.append(tf.keras.Input(shape=(4,),#4 info types
                                         name='dire_hero_info'+str(i)))
        x_dhi = dense_hero_info(hero_input[-1])
        #add dire hero tags to hero_input list and process with
        #    dense_tags layer
        hero_input.append(tf.keras.Input(shape=(6,),#6 tag types
                                         name='dire_hero_tags'+str(i)))
        x_dht = dense_hero_tags(hero_input[-1])
        #append output of hero layers to dire_out layer for summing later
        dire_hero_output.append(tf.keras.layers.concatenate([x_dh,x_dhi,x_dht]))
    
    #sum hero outputs to erase ordering information and get ave team vector
    rad_hero_sum = tf.keras.layers.Add()(rad_hero_output)
    dire_hero_sum = tf.keras.layers.Add()(dire_hero_output)    
    
    # Add a linear layer to map the average of the embedded vectors of each team
    #   to a single vector for each team
    x = tf.keras.layers.concatenate([rad_hero_sum,dire_hero_sum])
    x = tf.keras.layers.Dense(1, activation='linear')(x)

    #define skill input layer
    skill = tf.keras.Input(shape=(2,),name='skill_level')
    #concat skill input with output of hero layers
    x = tf.keras.layers.concatenate([x, skill])
    
    #Adds output layer with sigmoid activation for prediction
    win_predict = tf.keras.layers.Dense(1,activation='sigmoid')(x)
    model = tf.keras.Model(inputs=[hero_input,skill],
                           outputs=win_predict,name="radiant_win")
    tf.keras.utils.plot_model(model, 'multi_input_model.png', show_shapes=True)
    return model

In [21]:
#define the linear model with no feature columns
def train_linear_model_extra_info(optimizer, 
                                  vocab_len, 
                                  embedding_dims,
                                  batch_size,
                                  epochs,
                                  training_path,
                                  validation_path,
                                  compression_type='',
                                  temp_log_flag=True):
    '''Function to train simple keras linear model
    
    Args: 
        optimizer: A `tf.keras.optimizer` object to use for the model
        vocab_len: An `int` specifying the number of category of the categorical input
        embedding_dims: An `int` specifying the number of dimensions in the embedding layer
        batch_size: An `int` specifying batch size for each train step
        epochs: An `int` number of epochs to train for
        training_path: A `string` specifying training file
        validation_path: A `string` specifying validatoin file
        compression_type: A `string` specifying compression type
        temp_log_flag: A `bool` specifying whether to save logs in temp folder
    Returns:
        history: A `tf.keras.history` object that has loss and other metrics
    '''
    
    model = define_linear_model_extra_info(vocab_len,embedding_dims)
    model.compile(optimizer = optimizer,
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])

    ##get embedding weights for returning and saving
    #embedding_weights = model.layers[1].get_weights()[0]
    
    #get datasets from tfrecords and according to input_fn
    train_ds = _input_fn_extra_info(training_path,
                               batch_size=batch_size,
                               compression_type=compression_type)

    validation_ds= _input_fn_extra_info(validation_path,
                                   batch_size=batch_size,
                                   compression_type=compression_type)

    #allow logging in temp directory or directory to be included in git
    #  useful if tuning hyperparams or testing
    if temp_log_flag:
        logdir = os.path.join('..','..','..','tmp_log_dir',
                          datetime.now().strftime("lol_%Y%m%d-%H%M%S"))
    else:
        logdir = os.path.join('log_dir',
                          datetime.now().strftime("%Y%m%d-%H%M%S"))
        
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
    steps_per_epoch=50
    history = model.fit(train_ds, 
                        epochs = epochs,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=steps_per_epoch,
                        verbose=1,
                        validation_data=validation_ds,
                        callbacks=[tensorboard_callback]
                       )
    
    #save trained model. This can be used to retrain, predict, 
    #  or reloaded as a keras model to do more model things.
    #Use TF `SavedModel` format so that it can be used in serving
    model.save(os.path.join(logdir,'saved_model'),
                      save_format='tf')
    
    #save params for this training to a json in the logdir
    hyper = optimizer._hyper
    hyper_dict={}
    for key,value in hyper.items():
        hyper_dict[key]=value.numpy().item()
    training_params = dict({'model_type':'single_target_linear_with_embedding',
                            'steps':steps_per_epoch*epochs,
                            'embedding_dims':embedding_dims,
                            'batch_size':batch_size,
                            'optimizer':optimizer._name,
                            'hyper_parameters':hyper_dict})
    with open(os.path.join(logdir,'training_params.json'),'w') as fp:
        json.dump(training_params,fp)
    
    ##save embedding in numpy format for later use
    ##  no hero_vocab, but that should be ok
    #np.save(os.path.join(logdir,'embedding.npy'),embedding_weights)
    
    fig,ax = plt.subplots(1,1,figsize=(1.6*3,3))
    plt.plot(history.history['accuracy'])
    plt.xlabel('epochs (50 steps per)')
    plt.ylabel('accuracy')
    plt.show()
    
    return history

### train no feature column win model

In [11]:
training_file_novocab = os.path.join('LOL_data',
                             'mix_tier',
                             'lol_training_data',
                             'blue_win_extra_info_dota2_compat_lol_training_data.tfrecords')
validation_file_novocab = os.path.join('LOL_data',
                             'mix_tier',
                             'lol_validation_data',
                             'blue_win_extra_info_dota2_compat_lol_validation_data.tfrecords')

In [27]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
history = train_linear_model_extra_info(
            optimizer,
            vocab_len=len(champ_vocab),
            embedding_dims=1,
            batch_size=500, 
            epochs=50, 
            training_path=training_file_novocab, 
            validation_path=validation_file_novocab,
            compression_type='GZIP')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Train for 50 steps, validate for 50 steps
Epoch 1/50
 1/50 [..............................] - ETA: 26s

SystemError: <built-in function len> returned a result with an error set